In [1]:
import os
os.environ["TRANSFORMERS_CACHE"] = "/newdata/HJQ/Cultural_Tourism_Program/RAGDemo/models"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["OLLAMA_MODELS"] = "/newdata/HTY/BIO_LLM/DS_70b/models"
os.environ["OLLAF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HOME"] = "/newdata/HJQ/Cultural_Tourism_Program/RAGDemo/models"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

import sys
print(sys.executable)  # 应显示虚拟环境路径而非系统路径

/root/anaconda3/envs/deepseek/bin/python


In [1]:
import os
os.environ["TRANSFORMERS_CACHE"] = "/newdata/HJQ/Cultural_Tourism_Program/RAGDemo/models"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["OLLAMA_MODELS"] = "/newdata/HTY/BIO_LLM/DS_70b/models"
os.environ["OLLAF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HOME"] = "/newdata/HJQ/Cultural_Tourism_Program/RAGDemo/models"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

from transformers import AutoProcessor, Blip2ForConditionalGeneration,  Blip2ForImageTextRetrieval
import torch
from PIL import Image

# 加载模型和处理器
device = "cuda:0"
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b", cache_dir="/newdata/HJQ/Cultural_Tourism_Program/RAGDemo/models")
model = Blip2ForImageTextRetrieval.from_pretrained(
    "Salesforce/blip2-opt-2.7b", 
    torch_dtype=torch.float16,  # 半精度节省显存
    device_map=device,          # 自动分配设备（CPU/GPU）
    cache_dir="/newdata/HJQ/Cultural_Tourism_Program/RAGDemo/models"  # 指定缓存目录
)


/root/anaconda3/envs/deepseek/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/anaconda3/envs/deepseek/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.40it/s]
Some weights of Blip2ForImageTextRetrieval were not initialized from the model checkpoint at Salesforce/blip2-

In [2]:
image = Image.open("/newdata/HJQ/Cultural_Tourism_Program/RAGDemo/OneRAG/Tutorial/NativeRAG/dataset/imgs/R0000019.png")
question = "Question: What is the cat doing? Answer:"

In [3]:
inputs = processor(image, return_tensors="pt").to(device, torch.float16)
# inputs = processor(image, return_tensors="pt").to(device)
img = model.vision_model(**inputs)
img.last_hidden_state.shape

torch.Size([1, 257, 1408])

In [4]:
image_embeds = img[0]
image_attention_mask = torch.ones(image_embeds.size()[:-1], dtype=torch.long, device=image_embeds.device)

print(image_embeds)
print(image_embeds.shape)

print(image_attention_mask)
print(image_attention_mask.shape)
query_tokens = model.query_tokens.expand(image_embeds.shape[0], -1, -1)
query_outputs = model.qformer(
    query_embeds=query_tokens,
    encoder_hidden_states=image_embeds,
    encoder_attention_mask=image_attention_mask,
    return_dict=None,
)

tensor([[[ 0.1410, -0.0515, -0.3416,  ..., -0.3865,  0.0906,  0.4053],
         [ 0.4800,  0.2905, -0.0299,  ..., -0.2404,  0.8228,  0.4644],
         [ 1.0273, -0.6450, -0.4998,  ...,  0.4060, -0.1692,  0.3262],
         ...,
         [ 0.5498,  0.3965,  0.4319,  ...,  0.1715, -0.0894,  0.2898],
         [-1.1221,  1.1895, -0.2715,  ...,  0.6948, -0.8276,  0.8906],
         [ 0.3005,  0.0367, -0.1541,  ..., -0.3083,  0.8696,  0.5493]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<NativeLayerNormBackward0>)
torch.Size([1, 257, 1408])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [5]:
print(query_outputs)
image_embeds = query_outputs[0]
print(image_embeds.shape)
print(query_outputs.last_hidden_state.shape)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0061,  0.2229,  0.1544,  ..., -0.4864, -0.1867, -0.4073],
         [ 1.6960,  0.0082, -0.6006,  ..., -0.7425,  1.0083, -0.6608],
         [-0.9796,  0.1995, -0.1847,  ..., -1.3880, -0.4337,  1.0066],
         ...,
         [-0.8283, -0.4150, -0.0110,  ..., -0.3855,  0.2681,  0.7748],
         [-0.7986,  0.2922,  0.5896,  ..., -0.6854, -1.5820, -0.1256],
         [ 0.0854,  0.4348, -0.9331,  ..., -1.7016,  0.2088, -0.5576]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-6.1461e-03,  2.2294e-01,  1.5441e-01, -3.2191e-01, -2.3167e-01,
          6.3223e-01, -2.2285e-01, -6.6286e-01, -2.5129e-02,  3.7720e-01,
         -2.2187e-01, -1.3422e-01, -6.2565e-01,  2.9700e-01, -3.6429e-01,
          4.2409e-01,  5.7565e-01, -1.4399e-01,  7.5582e-01,  4.2788e-02,
          1.4999e-01, -4.3969e-01, -1.3227e-01,  6.3375e-02,  1.3900e+00,
          5.9420e-01,  9.2109e-01, -1.2041e-02, -2

In [6]:
tokenizer = processor.tokenizer
inputs = tokenizer(
    question,
    return_tensors="pt",
    truncation=True,
    max_length=32
)

input_ids, attention_mask = inputs.input_ids.to(device), inputs.attention_mask.to(device)



In [7]:
print(input_ids)
# query_embeds = model.embeddings(input_ids)

tensor([[    2, 45641,    35,   653,    16,     5,  4758,   608,   116, 31652,
            35]], device='cuda:0')


In [8]:
vocab_size = model.config.text_config.vocab_size  # BLIP-2 的文本编码器词汇量
assert input_ids.max() < vocab_size, "输入ID值超出词汇表范围"

In [16]:
with torch.no_grad():
    query_embeds = model.embeddings(input_ids)
    print(query_embeds.shape)
# query_embeds = model.embeddings(input_ids)
# print(query_embeds.shape)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
text_query_tokens = model.query_tokens.expand(input_ids.shape[0], -1, -1)  # 扩展标准查询向量
text_attention_mask = torch.ones(text_query_tokens.size()[:-1],)  # 创建文本查询掩码
combined_attention_mask = torch.cat([text_attention_mask, attention_mask], dim=1)  # 需拼接掩码

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
text_outputs = self.qformer(
    query_embeds=text_query_tokens,
    query_length=text_query_tokens.shape[1],  # 使用实际查询长度
    attention_mask=combined_attention_mask,
    return_dict=return_dict,
)


# query_tokens = model.query_tokens.expand(image_embeds.shape[0], -1, -1)
# query_attention_mask = torch.ones(query_tokens.size()[:-1], dtype=torch.long, device=query_tokens.device)
# attention_mask = torch.cat([query_attention_mask, attention_mask], dim=1)
# query_embeds = model.embeddings(
#             input_ids=input_ids,
#             query_embeds=query_tokens)

NameError: name 'self' is not defined